## Imports

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('..'))

import pso
import cpso
import ripso
import optimizationFuncs as optim
import benchmark as bm

## Reload (if necessary)

In [ ]:
from importlib import reload

pso = reload(pso)
cpso = reload(cpso)
ripso = reload(ripso)
optim = reload(optim)
bm = reload(bm)

## Benchmark

In [ ]:
bench_iters = 10
dims = 2

# Get benchmarker for rastrigin
bench = bm.benches['rastrigin']

##########################################################################################

# Vanilla PSO
opter1 = bm.optimizers['vanilla']
evaluator1 = bench(opter1, dims)
print(f"Benchmarking Vanilla PSO on {evaluator1}")
spec1 = evaluator1.eval(bench_iters)


##########################################################################################


# Chaotic Adaswarm
opter2 = bm.optimizers['adaswarm'].get_chaotic_swarm(
    {
        'init_cmap'    : {
            'name'      : 'tent',            # Initialisation map --> Tent
            'args'      : ()                 # Default Tent parameter --> 0.49999
        },
        
        'dyn_cmap'     : {
            'name'      : 'inverse',         # Chaotic map for r1, r2 --> Inverse Lyapunov Exponent Map
            'args'      : (1.414, )          # Custom Lyapunov Exponent
        }
    }
)
evaluator2 = bench(opter2, dims)
print(f"\nBenchmarking Chaotic Adaswarm on {evaluator2}")
spec2 = evaluator2.eval(bench_iters)

##########################################################################################

# Reverse-Informed PSO
opter3 = bm.optimizers['ripso']
evaluator3 = bench(opter3, dims)
print(f"\nBenchmarking Reverse-Informed PSO on {evaluator3}")
spec3 = evaluator3.eval(bench_iters)

## Printing Overall specs

In [ ]:
keys = [k for k in spec1['ospec'].keys() if k != 'conv_curves']
specs = {
    'vanilla'      : spec1['ospec'],
    'adaswarm'     : spec2['ospec'],
    'ripso'        : spec3['ospec']
}

for k in keys :
    for pso_type, spec in specs.items() :
        print("{:>10} {:>25} {:>25}".format(pso_type, k, spec[k]))
    print("\n", end="")

## Print Worst and Best Convergence Curve

In [ ]:
fig, ax = plt.subplots(1, 2, gridspec_kw={'width_ratios' : [1, 1]})
fig.set_figheight(5)
fig.set_figwidth(10)

ccb1, ccw1 = spec1['ospec']['conv_curves']
ccb2, ccw2 = spec2['ospec']['conv_curves']
ccb3, ccw3 = spec3['ospec']['conv_curves']

ax[0].plot(np.arange(len(ccb1)), ccb1, 'r', label='Vanilla')
ax[0].plot(np.arange(len(ccb2)), ccb2, 'b', label='Adaswarm')
ax[0].plot(np.arange(len(ccb3)), ccb3, 'k', label='Reverse-Informed')
ax[0].legend()

ax[1].plot(np.arange(len(ccw1)), ccw1, 'r', label='Vanilla')
ax[1].plot(np.arange(len(ccw2)), ccw2, 'b', label='Adaswarm')
ax[1].plot(np.arange(len(ccw3)), ccw3, 'k', label='Reverse-Informed')
ax[1].legend()